In [1]:
import os
import json
from glob import glob

def load_reports(base_dir="../data/DITrans-EMNLP/political_report/jsons/"):
    json_paths = glob(os.path.join(base_dir, "*.json"))
    items = []

    for path in sorted(json_paths):
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        items.append((os.path.basename(path), data))

    return items

items = load_reports()

print("Loaded", len(items), "files")

Loaded 1397 files


In [2]:
!export HF_HUB_OFFLINE=1
!export TRANSFORMERS_OFFLINE=1

In [2]:
from transformers import pipeline


translator = pipeline("translation", model="facebook/nllb-200-distilled-600M", src_lang="eng_Latn", tgt_lang="mal_Mlym")
translator(["Hello, how are you?", "This is a test sentence."])

Device set to use cuda:0


[{'translation_text': 'ഹലോ, എങ്ങിനെ പോകുന്നു?'},
 {'translation_text': 'ഇത് ഒരു പരീക്ഷണ വാക്യമാണ്.'}]

In [4]:
docs = []
for filename, data in items[:10]:
    sentences = {}
    for obj in data.get("objects", []):
        for sen in obj.get("sentences", []):
            info = sen.get("line_info", {})
            sen_id = sen.get("sen_id")
            src = info.get("text_gt")
            tgt = info.get("text_trans_ref")
            if sen_id in sentences.keys():
                if src[0] != ' ':
                    src = ' ' + src
                sentences[sen_id]['src'] += src
            else:
                sentences[sen_id] = {
                    'src': src,
                    'tgt': tgt,
                    'sen_id': sen_id,
                }

    docs.append({
        "sentences": sentences,
    })

docs[1]

{'sentences': {1: {'src': 'For more information on this publication, visit www.rand.org/t/RR2442',
   'tgt': '有关本出版物的更多信息，请访问www.rand.org/t/RR2442',
   'sen_id': 1},
  2: {'src': 'Published by the RAND Corporation, Santa Monica, Calif.',
   'tgt': '兰德公司出版，加州圣莫尼卡',
   'sen_id': 2},
  3: {'src': '©Copyright 2018 RAND Corporation RAND is a registered trademark.',
   'tgt': '©版权所有2018年兰德公司兰德是注册商标。',
   'sen_id': 3},
  4: {'src': 'Limited Print and Electronic Distribution Rights',
   'tgt': '有限的印刷和电子发行权',
   'sen_id': 4},
  5: {'src': 'This document and trademark(s) contained herein are protected by law.',
   'tgt': '本文件和本文件所载商标（或）受法律保护。',
   'sen_id': 5},
  6: {'src': 'This representation of RAND intellectual property is provided for noncommercial use only.',
   'tgt': '兰德公司知识产权的这种表述仅供非商业用途。',
   'sen_id': 6},
  7: {'src': 'Unauthorized posting of this publication online is prohibited.',
   'tgt': '禁止未经授权在网上发布本出版物。',
   'sen_id': 7},
  8: {'src': 'Permission is given to duplicate this docu

In [19]:
# translator = pipeline("translation", model="facebook/nllb-200-3.3B", src_lang="eng_Latn", tgt_lang="zho_Hans")
translator = pipeline("translation", model="facebook/nllb-200-distilled-600M", src_lang="eng_Latn", tgt_lang="zho_Hans")


for doc in docs:
    sources = [doc['sentences'][sen_id]['src'] for sen_id in doc['sentences'].keys()]
    print(sources)
    targets = sources = [doc['sentences'][sen_id]['tgt'] for sen_id in doc['sentences'].keys()]
    translated = translator(sources)
    for i, sen_id in enumerate(doc['sentences'].keys()):
        doc['sentences'][sen_id]['translated2'] = translated[i]['translation_text']

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
Device set to use cuda:0


['RAND', 'JUSTICE,INFRASTRUCTURE,AND ENVIRONMENT', 'Is Climate Restoration an Appropriate Climate Policy  Goal？', 'Robert J. Lempert,Giacomo Marangoni ,Klaus Keller,Jessica Duke', 'Prepared for the RAND Frederick S. Pardee Center for Long Range Global Policy and the Future Human Condition']
['For more information on this publication, visit www.rand.org/t/RR2442', 'Published by the RAND Corporation, Santa Monica, Calif.', '©Copyright 2018 RAND Corporation RAND is a registered trademark.', 'Limited Print and Electronic Distribution Rights', 'This document and trademark(s) contained herein are protected by law.', 'This representation of RAND intellectual property is provided for noncommercial use only.', 'Unauthorized posting of this publication online is prohibited.', 'Permission is given to duplicate this document for personal use only, as long as it is unaltered and complete.', 'Permission is required from RAND to reproduce, or reuse in another form, any of its research documents for c

In [24]:
import sacrebleu

refs = []
preds = []

for doc in docs:
    pred, ref = [], []
    for sen_id, sen in doc["sentences"].items():
        pred.append(sen["translated2"])
        ref.append(sen["tgt"])
    preds.append(" ".join(pred))
    refs.append(" ".join(ref))

# for doc in docs[:1]:
#     for sen_id, sen in doc["sentences"].items():
#         preds.append(sen["translated2"])
#         refs.append(sen["tgt"])

# sacrebleu expects: 
#   predictions = list[str] 
#   references = list[list[str]]  (list of refs)
bleu = sacrebleu.corpus_bleu(preds, [refs], tokenize='char')

print("BLEU:", bleu.score)

BLEU: 69.95654967601108


In [20]:
docs[0]

{'sentences': {1: {'src': 'RAND',
   'tgt': '兰德公司',
   'sen_id': 1,
   'translated': '兰德公司',
   'translated2': '兰德公司'},
  2: {'src': 'JUSTICE,INFRASTRUCTURE,AND ENVIRONMENT',
   'tgt': '正义、基础设施与环境',
   'sen_id': 2,
   'translated': '正义,基础设施与环境',
   'translated2': '正义,基础设施与环境'},
  3: {'src': 'Is Climate Restoration an Appropriate Climate Policy  Goal？',
   'tgt': '恢复气候是一个适当的气候政策目标吗？',
   'sen_id': 3,
   'translated': '恢复气候是一个适当的气候政策目标吗?',
   'translated2': '恢复气候是适当的气候政策目标吗?'},
  4: {'src': 'Robert J. Lempert,Giacomo Marangoni ,Klaus Keller,Jessica Duke',
   'tgt': '罗伯特·J·兰佩尔，贾科莫·马兰戈尼，克劳斯·凯勒，杰西卡·杜克',
   'sen_id': 4,
   'translated': '罗伯特·J·兰佩尔,贾科莫·马兰戈尼,克劳斯·凯勒,杰西卡·杜克',
   'translated2': '罗伯特·J·兰佩尔,贾科莫·马兰戈尼,克劳斯·凯勒,杰西卡·杜克'},
  5: {'src': 'Prepared for the RAND Frederick S. Pardee Center for Long Range Global Policy and the Future Human Condition',
   'tgt': '为兰德弗雷德里克-S-帕迪长期全球政策和未来人类状况中心准备',
   'sen_id': 5,
   'translated': '为兰德弗雷德里克-S-帕迪长期全球政策和未来人类状况中心准备',
   'translated2': '为兰德弗雷德里克-S-帕迪长期

In [21]:
import sacrebleu

doc = {
 'sentences': {
  1: {'src': 'RAND','tgt': '兰德公司','translated': '兰德公司'},
  2: {'src': 'JUSTICE,INFRASTRUCTURE,AND ENVIRONMENT','tgt': '正义、基础设施与环境','translated': '正义,基础设施与环境'},
  3: {'src': 'Is Climate Restoration an Appropriate Climate Policy  Goal？','tgt': '恢复气候是一个适当的气候政策目标吗？','translated': '恢复气候是一个适当的气候政策目标吗?'},
  4: {'src': 'Robert J. Lempert,Giacomo Marangoni ,Klaus Keller,Jessica Duke','tgt': '罗伯特·J·兰佩尔，贾科莫·马兰戈尼，克劳斯·凯勒，杰西卡·杜克','translated': '罗伯特·J·兰佩尔,贾科莫·马兰戈尼,克劳斯·凯勒,杰西卡·杜克'},
  5: {'src': 'Prepared for the RAND Frederick S. Pardee Center for Long Range Global Policy and the Future Human Condition','tgt': '为兰德弗雷德里克-S-帕迪长期全球政策和未来人类状况中心准备','translated': '为兰德弗雷德里克-S-帕迪长期全球政策和未来人类状况中心准备'}
 }
}

preds = []
refs = []

for sen in doc['sentences'].values():
    preds.append(sen['translated2'])
    refs.append(sen['tgt'])

bleu = sacrebleu.corpus_bleu(preds, [refs], tokenize='char')
print("BLEU:", bleu.score)


KeyError: 'translated2'